<a href="https://colab.research.google.com/github/AJamal27891/Recommendation_system/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip /content/drive/MyDrive/data.zip 

Archive:  /content/drive/MyDrive/data.zip
replace data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/events.json        
  inflating: __MACOSX/data/._events.json  
  inflating: data/meta.json          
  inflating: __MACOSX/data/._meta.json  


In [3]:
import pandas as pd 
events = pd.read_json(r'/content/data/events.json')

In [4]:
events.events.to_json(r'events.json')

In [ ]:
events = pd.read_json(r'events.json')

In [ ]:
events= events.T

In [ ]:
events

In [ ]:
meta = pd.read_json(r'/content/data/meta.json')

In [ ]:
meta.meta.to_json(r'meta.json')
meta = pd.read_json(r'meta.json').T
meta

In [ ]:
events.describe()

In [ ]:
meta.describe()

In [ ]:
data = events.merge(meta,left_on='productid',right_on='productid')
data.describe().T

In [ ]:
data.info()

In [ ]:
# calculate freq patterns data
fp_data = data[['sessionid','productid']]
fp_data = fp_data.groupby(['sessionid','productid']).size().unstack(fill_value=0).astype(bool)


In [ ]:
fp_data.head(5)

In [ ]:
!pip install mlxtend==0.17.0

In [ ]:
import mlxtend
print(mlxtend.__version__)

In [ ]:
from mlxtend.frequent_patterns import fpgrowth 
fb_data = fpgrowth(fp_data, min_support=0.00002, use_colnames=True,max_len=2)

In [ ]:
from mlxtend.frequent_patterns import association_rules
fb_rules = association_rules(fb_data, metric="confidence", min_threshold=0.50)

In [ ]:
fb_rules.describe()

In [ ]:
fb_rules.lift.plot(kind='hist')

In [ ]:
fb_rules.confidence.plot(kind='hist')

In [ ]:
fb_rules.columns

In [ ]:
fb_rules = fb_rules[['antecedents','consequents','confidence']]
fb_rules.antecedents  = fb_rules.antecedents.apply(lambda x:list(x)[0]).astype("unicode")
fb_rules.consequents  = fb_rules.consequents.apply(lambda x:list(x)[0]).astype("unicode")

In [ ]:
#lift and right tabel 
fb_rules

In [ ]:
#merging 
fb_merged = fb_rules.merge(meta,left_on='antecedents',right_on='productid',suffixes=(None,"_left"))
fb_merged = fb_merged.merge(meta,left_on='consequents',right_on='productid',suffixes=(None,"_right"))

fb_merged

In [ ]:
features_list = list(fb_merged.columns)
features_list.remove('confidence')

In [ ]:
features = fb_merged[features_list]
features

In [ ]:
fb_merged.info()

In [ ]:
fb_merged.values[0]

In [ ]:
features.values

In [ ]:
#vectorizing using tfidf
corpus_list = []
for seq in features.values:
  try:
    x = ' '.join(seq)
  except :
    pass 
  corpus_list.append(x)

In [ ]:
#split test and training 
test_size = .2 
train_features = corpus_list[:int(len(corpus_list)*(1-test_size))]
train_labels = fb_merged.confidence.values[:int(len(corpus_list)*(1-test_size))]

In [ ]:
len(train_features) - len(train_labels)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X= vectorizer.fit_transform(train_features)

In [ ]:
X

In [ ]:
len(vectorizer.get_feature_names())

In [ ]:
encoded = vectorizer.transform(corpus_list[0].split())

In [ ]:
decoded= vectorizer.inverse_transform(encoded)

In [ ]:
for x in decoded:
  print(x)

In [ ]:
train_features = [vectorizer.transform(train_features)]

In [ ]:
X

In [ ]:
train_labels[0]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import TruncatedSVD
pca = TruncatedSVD(350)
X = pca.fit_transform(X)
regressor = RandomForestRegressor()

In [ ]:
import numpy as np
sum(pca.explained_variance_ratio_)

In [ ]:
X[0]

In [ ]:
regressor.fit(X, train_labels)

In [ ]:
features_test = corpus_list[len(X):]
features_test = vectorizer.transform(features_test)
features_test = pca.transform(features_test)
labels_test = fb_merged.confidence.values[len(X):]
prediction = regressor.predict(features_test)


In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(labels_test, prediction, squared=False)

In [ ]:
rmse